In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np

In [3]:
from ensembles import DiversityEnsembleClassifier as DEC
from ensembles import GeneticEnsembleClassifier as GEC
from ensembles import RandomClassifier as RC
from ensembles import MajorityClassifier as MC

### Loading datasets

In [ ]:
breast = datasets.load_breast_cancer()
iris   = datasets.load_iris()
wine   = datasets.load_wine()

In [52]:
yeast  = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data', header=None, delim_whitespace=True)
le = LabelEncoder()
yeast.iloc[:, -1] = le.fit_transform(yeast.iloc[:, -1])
yeast.iloc[:, 0] = le.fit_transform(yeast.iloc[:, 0])

In [ ]:
spam   = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', header=None)

In [ ]:
iono   = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data', header=None)
iono.iloc[:, -1] = le.fit_transform(iono.iloc[:, -1])

In [ ]:
captcha = pd.read_csv('dados_captcha.csv')
captcha.iloc[:, -1] = le.fit_transform(captcha.iloc[:, -1])

In [22]:
letters = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data', header=None)
letters = letters[(letters.iloc[:, 0] == 'J')|(letters.iloc[:, 0] == 'L')|(letters.iloc[:, 0] == 'K')]
letters.iloc[:, 0] = le.fit_transform(letters.iloc[:, 0])

In [36]:
image = pd.read_csv('image_segmentation.csv', header=None)
image.iloc[:, 0] = le.fit_transform(image.iloc[:, 0])

In [41]:
pendigits = pd.read_csv('pendigits.csv', header=None)
pendigits = pendigits[(pendigits.iloc[:, -1] == 9)|(pendigits.iloc[:, -1] == 3)|(pendigits.iloc[:, -1] == 8)]

In [61]:
waveform = pd.read_csv('waveform-+noise.data', header=None)

In [67]:
vehicles = pd.read_csv('vehicles.data',sep=' ', header=None)
vehicles = vehicles.iloc[:, :-1]
vehicles.iloc[:, -1] = le.fit_transform(vehicles.iloc[:, -1])

In [64]:
captcha.shape

(2788, 78)

<h3>Testing method</h3>

In [6]:
def test_algorithm(algorithm, hiperparam, data, target):
    accuracy, f1, precision, recall, auc = 0, 0, 0, 0, 0
    div = np.zeros(25)
    fit = np.zeros(25)
    for i in range(0, 10):
        print(i, end='')
        X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=i*10)
        try:classifier = algorithm(**hiperparam, random_state=i*10)#classifier.set_param(random_state=i*10)
        except: classifier = algorithm(**hiperparam)#classifier.random_state=i*10
        aux = classifier.fit(X_train, y_train)
        try: 
            div += aux[0]
            fit += aux[1]
        except: pass
        y_pred = classifier.predict(X_test)
        accuracy += accuracy_score(y_test, y_pred)
        try: f1 += f1_score(y_test, y_pred)
        except: pass
        try: precision += precision_score(y_test, y_pred)
        except: pass
        try: recall += recall_score(y_test, y_pred)
        except: pass
        try: auc += roc_auc_score(y_test, y_pred)
        except: pass
    print('Accuracy :', accuracy/10)
    if f1>0: print('F1-score :', f1/10)
    if precision>0: print('Precision:', precision/10)
    if recall>0: print('Recall   :', recall/10)
    if auc>0: print('ROC AUC  :', auc/10)
    return div/10, fit/10

In [69]:
def compare_results(data, target, size):
    n_samples = ((data.shape[0]*0.8 * 4) // 5)-4
    alg = {
                KNeighborsClassifier: {'n_neighbors':[1, n_samples]},
                RidgeClassifier: {'alpha':[1.0, 10.0],'max_iter':[10, 100]},
                SVC: {'C':[1, 1000],
                      'gamma':[0.0001, 0.001]
                      },
                DecisionTreeClassifier: {'min_samples_leaf':[1, n_samples], 'max_depth':[1, n_samples]},
                ExtraTreeClassifier: {'min_samples_leaf':[1, n_samples], 'max_depth':[1, n_samples]},
                GaussianNB: {},
                LinearDiscriminantAnalysis: {},
                QuadraticDiscriminantAnalysis: {},
                BernoulliNB: {},
                #GaussianProcessClassifier: {'max_iter_predict':[1, 50]},
                LogisticRegression:{'C':[1, 1000], 'max_iter':[100, 1000]},
                NearestCentroid:{},
                PassiveAggressiveClassifier:{'C':[1, 1000], 'max_iter':[100, 1000]},
                SGDClassifier: {'alpha':[1e-5, 1e-2], 'max_iter':[100, 1000]}
          }
    
    
    print('-'*60)
    print('Diversity-based Ensemble Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    dec = {'algorithms':alg, 'population_size':size, 'max_epochs':25}
    div, fit = test_algorithm(DEC, dec, data=data, target=target)
    print('DEC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
    
    print('-'*60)
    print('Genetic-based Ensemble Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    gec = {'algorithms':alg, 'population_size':size, 'max_epochs':25}
    test_algorithm(GEC, gec, data=data, target=target)
    print('GEC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')    
                   
    print('-'*60)
    print('Random Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    rc = {'algorithms':alg}
    test_algorithm(RC, rc, data=data, target=target)
    print('RC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
      
    print('-'*60)
    print('Majority Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    #mc = MC()
    test_algorithm(MC, {}, data=data, target=target)
    print('MC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
    
    print('-'*60)
    print('Random Forest')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    rf = {'n_estimators':size}
    test_algorithm(RandomForestClassifier, rf, data=data, target=target)
    print('RF done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
        
    print('-'*60)
    print('Ada Boost')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    ada = {'n_estimators':size}
    test_algorithm(AdaBoostClassifier, ada, data=data, target=target)
    print('Ada done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
       
    return div, fit

In [34]:
d10, f10   =compare_results(data=captcha.iloc[:, 0:-1].values, target=captcha.iloc[:, -1].values, size=10)
d25, f25   =compare_results(data=captcha.iloc[:, 0:-1].values, target=captcha.iloc[:, -1].values, size=25)
d50, f50 =compare_results(data=captcha.iloc[:, 0:-1].values, target=captcha.iloc[:, -1].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('captcha.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('captchaFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.7580645161290324
DEC done in:  397243.7 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.753584229390681
GEC done in:  372717.7 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.15089605734767025
RC done in:  478.9 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.0931899641577061
MC done in:  6.1 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accu

In [9]:
d25, f25   =compare_results(data=breast.data, target=breast.target, size=10)
d50, f50   =compare_results(data=breast.data, target=breast.target, size=25)
d100, f100 =compare_results(data=breast.data, target=breast.target, size =50)

plt.plot(range(1, 26), d25, label='10 cromossomos')
plt.plot(range(1, 26), d50, label='25 cromossomos')
plt.plot(range(1, 26), d100, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('breast2.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f25, label='10 cromossomos')
plt.plot(range(1, 26), f50, label='25 cromossomos')
plt.plot(range(1, 26), f100, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('breastFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9140350877192983
F1-score : 0.9356655339566602
Precision: 0.8862962146322158
Recall   : 0.9930788912579956
ROC AUC  : 0.8887538964017118
DEC done in:  8566.9 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.95
F1-score : 0.9601419839289264
Precision: 0.9557252641349689
Recall   : 0.9652811206353353
ROC AUC  : 0.9456808819867624
GEC done in:  8230.4 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.8
F1-score : 0.868738241336074
Precision: 0.7874143242546172
Recall   : 0.9857455579246623
ROC AUC  : 0.7325360545165641
RC done in:  9.2 ms
----------------------------

In [10]:
d25, f25 = compare_results(data=iris.data, target=iris.target, size=10)

d50, f50 = compare_results(data=iris.data, target=iris.target, size=25)

d100, f100 = compare_results(data=iris.data, target=iris.target, size=50)

plt.plot(range(1, 26), d25, label='10 cromossomos')
plt.plot(range(1, 26), d50, label='25 cromossomos')
plt.plot(range(1, 26), d100, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('iris.png')
plt.clf()
plt.close()


plt.plot(range(1, 26), f25, label='10 cromossomos')
plt.plot(range(1, 26), f50, label='25 cromossomos')
plt.plot(range(1, 26), f100, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('irisFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9166666666666667
DEC done in:  3471.7 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9766666666666668
GEC done in:  3238.2 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.7866666666666667
RC done in:  2.9 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.25000000000000006
MC done in:  1.5 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accuracy 

In [11]:
d25, f25 = compare_results(data=wine.data, target=wine.target, size=10)
d50, f50 = compare_results(data=wine.data, target=wine.target, size=25)
d100, f100 = compare_results(data=wine.data, target=wine.target, size=50)

plt.plot(range(1, 26), d25, label='10 cromossomos')
plt.plot(range(1, 26), d50, label='25 cromossomos')
plt.plot(range(1, 26), d100, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('wine.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f25, label='10 cromossomos')
plt.plot(range(1, 26), f50, label='25 cromossomos')
plt.plot(range(1, 26), f100, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('wineFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.8416666666666666
DEC done in:  4667.8 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9888888888888889
GEC done in:  4740.0 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.7555555555555556
RC done in:  5.0 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.38333333333333336
MC done in:  1.5 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accuracy 

In [57]:
d25, f25 = compare_results(data=yeast.iloc[:, 0:-1].values, target=yeast.iloc[:, -1].values, size = 10)
d50, f50 = compare_results(data=yeast.iloc[:, 0:-1].values, target=yeast.iloc[:, -1].values, size = 25)
d100, f100 = compare_results(data=yeast.iloc[:, 0:-1].values, target=yeast.iloc[:, -1].values, size = 50)

plt.plot(range(1, 26), d25, label='10 cromossomos')
plt.plot(range(1, 26), d50, label='25 cromossomos')
plt.plot(range(1, 26), d100, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('yeast.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f25, label='10 cromossomos')
plt.plot(range(1, 26), f50, label='25 cromossomos')
plt.plot(range(1, 26), f100, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('yeastFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.6336700336700336
DEC done in:  88118.1 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.6127946127946128
GEC done in:  83965.2 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.36094276094276095
RC done in:  195.9 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.3178451178451179
MC done in:  1.5 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accur

In [ ]:
d25, f25 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=10)
d50, f50 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=25)
d100, f100 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=50)

plt.plot(range(1, 26), d25, label='10 cromossomos')
plt.plot(range(1, 26), d50, label='25 cromossomos')
plt.plot(range(1, 26), d100, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('spam.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f25, label='10 cromossomos')
plt.plot(range(1, 26), f50, label='25 cromossomos')
plt.plot(range(1, 26), f100, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('spamFitness.png')
plt.clf()
plt.close()

In [71]:
d10, f10   =compare_results(data=iono.iloc[:, 0:-1].values, target=iono.iloc[:, -1].values, size=10)
d25, f25   =compare_results(data=iono.iloc[:, 0:-1].values, target=iono.iloc[:, -1].values, size=25)
d50, f50 =compare_results(data=iono.iloc[:, 0:-1].values, target=iono.iloc[:, -1].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('iono.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('ionoFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.8901408450704225
F1-score : 0.916158491188666
Precision: 0.8589726085104289
Recall   : 0.9835888907840127
ROC AUC  : 0.8589828080559124
DEC done in:  4982.0 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9394366197183096
F1-score : 0.9512058708771495
Precision: 0.9467223105030704
Recall   : 0.9565037523452158
ROC AUC  : 0.9305214730210091
GEC done in:  4982.1 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.7732394366197182
F1-score : 0.8319806373154425
Precision: 0.7883497462153495
Recall   : 0.9097758258002159
ROC AUC  : 0.7307113255985207
RC done in:  4.5 ms

In [33]:
d10, f10   =compare_results(data=letters.iloc[:, 1:].values, target=letters.iloc[:, 0].values, size=10)
d25, f25   =compare_results(data=letters.iloc[:, 1:].values, target=letters.iloc[:, 0].values, size=25)
d50, f50 =compare_results(data=letters.iloc[:, 1:].values, target=letters.iloc[:, 0].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('letters.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('lettersFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9202222222222222
DEC done in:  54700.2 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9977777777777777
GEC done in:  57012.3 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.8464444444444444
RC done in:  70.0 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.3251111111111111
MC done in:  1.5 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accurac

In [38]:
d10, f10   =compare_results(data=image.iloc[:, 1:].values, target=image.iloc[:, 0].values, size=10)
d25, f25   =compare_results(data=image.iloc[:, 1:].values, target=image.iloc[:, 0].values, size=25)
d50, f50 =compare_results(data=image.iloc[:, 1:].values, target=image.iloc[:, 0].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('image.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('imageFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.892857142857143
DEC done in:  9290.6 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.8952380952380953
GEC done in:  9205.6 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.6095238095238095
RC done in:  8.0 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.07619047619047618
MC done in:  1.3 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accuracy :

In [43]:
d10, f10   =compare_results(data=pendigits.iloc[:, 0:-1].values, target=pendigits.iloc[:, -1].values, size=10)
d25, f25   =compare_results(data=pendigits.iloc[:, 0:-1].values, target=pendigits.iloc[:, -1].values, size=25)
d50, f50 =compare_results(data=pendigits.iloc[:, 0:-1].values, target=pendigits.iloc[:, -1].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('pendigits.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('pendigitsFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9404423380726697
DEC done in:  102063.0 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.9968404423380726
GEC done in:  105792.0 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.8257503949447077
RC done in:  50.4 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.31721958925750393
MC done in:  1.6 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accu

In [47]:
d10, f10   =compare_results(data=waveform.iloc[:, 0:-1].values, target=waveform.iloc[:, -1].values, size=10)
d25, f25   =compare_results(data=waveform.iloc[:, 0:-1].values, target=waveform.iloc[:, -1].values, size=25)
d50, f50 =compare_results(data=waveform.iloc[:, 0:-1].values, target=waveform.iloc[:, -1].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('waveform.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('waveformFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.786
DEC done in:  314047.3 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.8698
GEC done in:  302541.4 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.7314999999999999
RC done in:  144.4 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.3291
MC done in:  5.6 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accuracy : 0.8138
RF done in:  151.5 ms
-

In [72]:
d10, f10   =compare_results(data=vehicles.iloc[:, 0:-1].values, target=vehicles.iloc[:, -1].values, size=10)
d25, f25   =compare_results(data=vehicles.iloc[:, 0:-1].values, target=vehicles.iloc[:, -1].values, size=25)
d50, f50 =compare_results(data=vehicles.iloc[:, 0:-1].values, target=vehicles.iloc[:, -1].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('vehicles.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('vehiclesFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.7558823529411764
DEC done in:  27641.0 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
0123456789Accuracy : 0.841764705882353
GEC done in:  24577.9 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
0123456789Accuracy : 0.5111764705882352
RC done in:  25.6 ms
------------------------------------------------------------
Majority Classifier
------------------------------------------------------------
0123456789Accuracy : 0.22941176470588234
MC done in:  1.6 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
0123456789Accurac